In [ ]:
#read in pandas titanic dataset from kaggle.com
import pandas as pd
train = pd.read_csv('train.csv')
train_shape = train.shape
test = pd.read_csv("test.csv")
test_shape = test.shape

In [ ]:
#explore survival by sex
print(train["Sex"].describe())
sex_pivot = train.pivot_table(index="Sex",values="Survived")
sex_pivot.plot.bar()
plt.show()

In [ ]:
#explore survival by Pclass
import matplotlib.pyplot as plt
print(train["Pclass"].describe())
pclass_pivot = train.pivot_table(index = 'Pclass', values = 'Survived')
pclass_pivot.plot.bar()
plt.show

In [ ]:
#explore survival ba age
print(train["Age"].describe())

#fill the missing values and categorizing the age
def process_age(df,cut_points,label_names):
    df["Age"] = df["Age"].fillna(-0.5)
    df["Age_categories"] = pd.cut(df["Age"],cut_points,labels=label_names)
    return df

cut_points = [-1,0,5,12,18,35,60,100]
label_names = ['Missing','Infant','Child','Teenager','Young Adult','Adult','Senior']

train = process_age(train,cut_points,label_names)
test = process_age(test,cut_points,label_names)

train_pivot = train.pivot_table(index = "Age_categories", values = "Survived")
train_pivot.plot.bar()
plt.show()

In [ ]:
# convert nonnumeric values 
def create_dummies(df,column_name):
    dummies = pd.get_dummies(df[column_name],prefix=column_name)
    df = pd.concat([df,dummies],axis=1)
    return df

train = create_dummies(train,"Pclass")
test = create_dummies(test,"Pclass")
train = create_dummies(train , "Sex")
test = create_dummies(test, "Sex")
train = create_dummies(train,"Age_categories")
test = create_dummies(test,"Age_categories")

In [ ]:
#Using Logistic Regression
columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']

from sklearn.linear_model import LogisticRegression
lr = LogisticRegression()

lr.fit(train[columns],train['Survived'])

In [ ]:
#train test split
holdout = test # from now on we will refer to this
               # dataframe as the holdout data

from sklearn.model_selection import train_test_split

columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']
train_X,test_X,train_y,test_y = train_test_split(train[columns],train['Survived'],test_size = 0.2, random_state = 0)

In [ ]:
#fiting and testing the model
from sklearn.metrics import accuracy_score
lr = LogisticRegression()
lr.fit(train_X,train_y)
predictions = lr.predict(test_X)
accuracy = accuracy_score(test_y,predictions)
print(accuracy)

In [ ]:
#using k-fold cross validation to train the model
from sklearn.model_selection import cross_val_score
import numpy as np
lr = LogisticRegression()
scores = cross_val_score(lr,all_X,all_y,cv = 10)
accuracy = np.mean(scores)
print(scores,accuracy)


In [ ]:
#make predictions on holdout dataset
columns = ['Pclass_1', 'Pclass_2', 'Pclass_3', 'Sex_female', 'Sex_male',
       'Age_categories_Missing','Age_categories_Infant',
       'Age_categories_Child', 'Age_categories_Teenager',
       'Age_categories_Young Adult', 'Age_categories_Adult',
       'Age_categories_Senior']
lr = LogisticRegression()
lr.fit(all_X,all_y)
holdout_predictions = lr.predict(holdout[columns])

In [ ]:
#create submission file
holdout_ids = holdout["PassengerId"]
submission_df = {"PassengerId": holdout_ids,
                 "Survived": holdout_predictions}
submission = pd.DataFrame(submission_df)
submission.to_csv('submission.csv',index = False)